In [1]:
# TASK 1: Analysis of step-one performance retrieval.
# First thing, start easy: build two-indexes, evaluate them top-m retrievals
# Hit-once and Hit-all. Then see other metrics like precision & recall + Roni's analysis.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cleaning sample_data directory
!rm -r sample_data

In [3]:
!ls drive/My\ Drive/touche-2022-prototyping

initial-retrieval-prototyping-and-analysis.ipynb
mon-duo-retrieval-prototyping-and-analysis.ipynb
topics-task2-51-100.xml
touche-task2-51-100-relevance.qrels
touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl
Untitled


In [ ]:
# One Time Exection.
# extracting files and converting them to 'tar.gz' files.
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl.gz
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl.gz

In [5]:
# installing linux related stuff for pyserini
# removing error "ImportError: No module named '_swigfaiss"
# Reference Link: https://github.com/facebookresearch/faiss/issues/821
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [6]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

     |████████████████████████████████| 84.6 MB 141 kB/s 
     |████████████████████████████████| 13.5 MB 19.7 MB/s 
     |████████████████████████████████| 2.0 MB 43.6 MB/s 
     |████████████████████████████████| 5.2 MB 33.6 MB/s 
     |████████████████████████████████| 1.2 MB 43.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 3.8 MB 39.4 MB/s 
     |████████████████████████████████| 6.0 MB 18.3 MB/s 
     |████████████████████████████████| 188 kB 48.3 MB/s 
     |████████████████████████████████| 10.1 MB 23.3 MB/s 
     |████████████████████████████████| 451 kB 47.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 47.8 MB/s 
     |████████████████████████████████| 653 kB 10.8 MB/s 
     |████████████████████████████████| 895 kB 52.2 MB/s 
     |████████████████████████████████| 6.5 MB 37.8 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████

In [17]:
# import statements
import pandas as pd
import jsonlines
from xml.dom import minidom
import numpy as np

In [ ]:
# original corpus
# corpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl'))

In [8]:
# query expansion corpus
qcorpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl'))

In [9]:
# printing demo query
qcorpus_list[0]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>",
 'id': 'clueweb12-0000tw-00-14115___1'

In [10]:
!mkdir sample_collection_jsonl
!mkdir indexes
!mkdir indexes/sample_collection_jsonl

In [11]:
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in qcorpus_list:
    output.write({
        'id': vals['id'],
        'contents': vals['contents']
    })

In [12]:
!head -10 sample_collection_jsonl/documents.jsonl

{"id": "clueweb12-0000tw-00-14115___1", "contents": "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>"}
{"id": "clueweb12-0000tw-00-14115___10", "contents": "By insisting on upholding affirmative action in colle

In [29]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/sample_collection_jsonl \
  --index /content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-04-02 13:00:33,191 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-02 13:00:33,198 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-02 13:00:33,198 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-02 13:00:33,199 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: /content/sample_collection_jsonl
2022-04-02 13:00:33,200 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-02 13:00:33,205 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-02 13:00:33,206 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-04-02 13:00:33,206 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-02 13:00:33,207 INFO  [main] index.IndexC

In [14]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0_Lucene80_0.dvd  _0_Lucene84_0.tim  _0.nvm  _0.tvx
_0.fdt	_0_Lucene80_0.dvm  _0_Lucene84_0.tip  _0.si   segments_1
_0.fdx	_0_Lucene84_0.doc  _0_Lucene84_0.tmd  _0.tvd  write.lock
_0.fnm	_0_Lucene84_0.pos  _0.nvd	      _0.tvm


In [15]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('indexes/sample_collection_jsonl')
hits = searcher.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
# Attempted tuning BM25 and RM3, nothing has changed the document ranking below.
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 16.55650
2 clueweb12-0605wb-29-30060___11 15.76430
3 clueweb12-1709wb-12-07226___7 15.69540
4 clueweb12-0605wb-29-30060___2 15.21990
5 clueweb12-0904wb-33-14589___7 15.14270
6 clueweb12-1400wb-06-30596___4 15.02100
7 clueweb12-1411wb-25-14634___8 14.83110
8 clueweb12-0605wb-29-30060___10 14.75490
9 clueweb12-1014wb-84-07457___12 14.68790
10 clueweb12-1400tw-67-08199___28 14.26160
11 clueweb12-0306wb-63-29079___5 14.16490
12 clueweb12-0605wb-57-17257___8 13.69220


In [16]:
# Grab the raw text:
print(hits[0].raw)
print('\n')
# Grab the raw Lucene Document:
print(hits[0].lucene_document)

{
  "id" : "clueweb12-1400wb-06-30596___3",
  "contents" : "There is a limit to how angry someone gets...even the Hulk and even at his angriest he wouldn't be as strong as Superman. No one really knows how strong the man of steel is and perhaps neither does he! Many wrongly think that just because Hulk is bigger and looks stronger than it means he is stronger. Superman can move planets...need I say more! Others believe Hulk would win because they're likely Marvel fan boys. Superman has defeated powerful opponents like Darkseid,Mongrel,Parasite,Doomsday,and General Zod;several of whom could likely take Hulk! Superman has more varied powers and because he's smarter can be more resourceful in using them. Speaking of intelligence,how the hell can anyone think that Superman isn't smarter than a human,even one as brilliant as Dr.Bruce Banner. Superman/Kal-El is not only the son of Jor-El,Krypton's most brilliant scientist,but he's outsmarted Lex Luthor(one of the smartest people on earth)and

In [18]:
# define function for loading all the topics from the topics files.
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title"])
  return parsed

In [19]:
topics_2021 = parse_xml("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-51-100.xml")
topics_2021.head()

,Number,Title
0,51,"What is better at reducing fever in children, ..."
1,52,What are the best rice cookers?
2,53,Should I buy steel or ceramic knives?
3,54,Is morning or afternoon sun the best for fruit...
4,55,"What is better for back pain, chiropractic the..."


In [121]:
# Run loop for topics_2021 data, store retrieved in order of appearance m=275
solution_dict = {} # topic_id, corresponding_document list.
for id_, q_ in zip(topics_2021['Number'], topics_2021['Title']):
    hits = searcher.search(q_, k=500)
    d_list = []
    for h_ in hits:
        d_= h_.docid.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)

    solution_dict[id_] = d_list

In [102]:
searcher_opt = LuceneSearcher('indexes/sample_collection_jsonl')
searcher_opt.set_bm25(1.2, 0.68)
hits = searcher_opt.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 15.37180
2 clueweb12-1709wb-12-07226___7 14.85930
3 clueweb12-0605wb-29-30060___11 14.84560
4 clueweb12-0605wb-29-30060___10 13.93880
5 clueweb12-0605wb-29-30060___2 13.85450
6 clueweb12-1400wb-06-30596___4 13.81460
7 clueweb12-0904wb-33-14589___7 13.78280
8 clueweb12-1411wb-25-14634___8 13.60230
9 clueweb12-1014wb-84-07457___12 13.35340
10 clueweb12-1400tw-67-08199___28 13.04680
11 clueweb12-0306wb-63-29079___5 12.96960
12 clueweb12-0605wb-57-17257___8 12.88170


In [122]:
# Run loop for topics_2021 data, store retrieved in order of appearance m=275
solution_dict_opt = {} # topic_id, corresponding_document list.
for id_, q_ in zip(topics_2021['Number'], topics_2021['Title']):
    hits = searcher_opt.search(q_, k=500)
    d_list = []
    for h_ in hits:
        d_= h_.docid.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)

    solution_dict_opt[id_] = d_list

In [32]:
# creating new qrel formatted file which only has the matching doc-ids from corpus.
jsonObj = pd.read_json(path_or_buf='drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl', lines=True)
jsonObj["old_id"]=jsonObj["id"].str.split("___",expand=True)[0]
documents_2021=jsonObj.groupby(by="old_id")["contents"].apply(" ".join).reset_index()
del jsonObj
documents_2021.head()

,old_id,contents
0,clueweb12-0000tw-00-14115,Do Asian-Americans Face Bias in Admissions at ...
1,clueweb12-0000tw-00-15738,Big Data Analytics a Key Enabler for Social CR...
2,clueweb12-0000tw-00-17905,Best Bets: Fern brings magic to Milford - Delm...
3,clueweb12-0000tw-01-02230,Python Data Analysis Library — pandas: Python ...
4,clueweb12-0000tw-01-15084,MILLARWORLD WEEK: Mark Millar’s Master Plan | ...


In [33]:
rel_2021 = pd.read_csv('/content/drive/MyDrive/touche-2022-prototyping/touche-task2-51-100-relevance.qrels',
                        sep="\s+", 
                        names=["qid", "no", "doc", "rel"])
rel_2021 = rel_2021.astype(str)
rel_2021.head()

,qid,no,doc,rel
0,54,0,clueweb12-0205wb-64-11095,0
1,54,0,clueweb12-0501wb-64-06459,1
2,54,0,clueweb12-0207wb-30-15337,0
3,54,0,clueweb12-0906wb-38-27123,0
4,54,0,clueweb12-0907wb-55-11510,0


In [36]:
# printing the data types
print(topics_2021.dtypes)
print('\n')
print(rel_2021.dtypes)

Number    object
Title     object
dtype: object


qid    object
no     object
doc    object
rel    object
dtype: object


In [38]:
percentage=[]
not_common_docs=[]
for qid in topics_2021["Number"]:
    relevant=rel_2021.loc[rel_2021["qid"]==qid]["doc"]
    intersection=set(relevant) & set(documents_2021["old_id"])
    not_common=list(set(relevant) -set(documents_2021["old_id"]))
    percent=len(intersection)/len(relevant)
    not_common_docs.append(not_common)
    percentage.append({"title_id":qid,"Percentage":percent,"Not_common":not_common})
df=pd.DataFrame(percentage)
df.head()

,title_id,Percentage,Not_common
0,51,0.809524,"[clueweb12-0000wb-65-16624, clueweb12-0005wb-4..."
1,52,0.909091,"[clueweb12-1806wb-58-25301, clueweb12-1303wb-1..."
2,53,0.947368,"[clueweb12-0606wb-89-10483, clueweb12-1503wb-1..."
3,54,0.933333,"[clueweb12-0906wb-38-27122, clueweb12-0503wb-8..."
4,55,0.975000,[clueweb12-1509wb-36-29085]


In [39]:
not_common_docs=list(np.concatenate(not_common_docs).flat)
new_rel_2021=rel_2021[~rel_2021["doc"].isin(not_common_docs)]
new_rel_2021.to_csv("/content/drive/MyDrive/touche-2022-prototyping/touche_results_2021.csv")
# documents_2021.to_csv("/content/drive/MyDrive/touche-2022-prototyping/documents_2021.csv")

In [40]:
# verifying the irrelevant doc-id elimination operation.
percentage=[]
not_common_docs=[]
for qid in topics_2021["Number"]:
    relevant=new_rel_2021.loc[new_rel_2021["qid"]==qid]["doc"]
    intersection=set(relevant) & set(documents_2021["old_id"])
    not_common=list(set(relevant) -set(documents_2021["old_id"]))
    percent=len(intersection)/len(relevant)
    not_common_docs.append(not_common)
    percentage.append({"title_id":qid,"Percentage":percent,"Not_common":not_common})
print(percentage)
# successfully the additionals documents are eliminated.

[{'title_id': '51', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '52', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '53', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '54', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '55', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '56', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '57', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '58', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '59', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '60', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '61', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '62', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '63', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '64', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '65', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '66', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '67', 'Percentage': 1.0, 'Not_common': []}, {'title_id': '68', 'Percentage

In [41]:
new_rel_2021.head()

,qid,no,doc,rel
0,54,0,clueweb12-0205wb-64-11095,0
1,54,0,clueweb12-0501wb-64-06459,1
2,54,0,clueweb12-0207wb-30-15337,0
3,54,0,clueweb12-0906wb-38-27123,0
4,54,0,clueweb12-0907wb-55-11510,0


In [98]:
from collections import defaultdict
ground_truth_dict = defaultdict(list)
for i_, d_, x_ in zip(new_rel_2021['qid'], new_rel_2021['doc'], new_rel_2021['rel']):
    if int(x_) > 0:
        i_ = str(i_)
        d_ = str(d_)
        ground_truth_dict[i_].append(d_)

In [123]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 50
per_comm_avg = 0
for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.52
Average common ratio: 0.9183


In [124]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 50
per_comm_avg = 0
for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.5
Average common ratio: 0.9196


In [ ]:
# Result summarization and A case for query expansion.
# Measuring on an average what's the document retrieval coverage.
# 1. k = 500, opt
# Hit one: 1.0
# Hit all: 0.5
# Average common ratio: 0.9196

# 2. k = 786, opt
#    Hit one: 1.0
#    Hit all: 0.64
#    Average common ratio: 0.944

# 3. k = 1000, opt 
#    Hit one: 1.0
#    Hit all: 0.7
#    Average common ratio: 0.9563

# 4. k = 1200 
#    Hit one: 1.0
#    Hit all: 0.74
#    Average common ratio: 0.9726

# 5. k = 1375
#    Hit one: 1.0
#    Hit all: 0.78
#    Average common ratio: 0.9745

# 6. k = 1500, opt
#    Hit one: 1.0
#    Hit all: 0.78
#    Average common ratio: 0.9752